In [1]:
import DataLoader
import importlib
import pandas as pd
import CoverageHelper
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
importlib.reload(DataLoader)
test_graph_df = DataLoader.load_csv_test_graphs()
stat_graph_df = DataLoader.load_csv_stat_graph(False)
stat_trans_graph_df = DataLoader.load_csv_stat_graph(True)

Number of apps: 
1
Total entries:
5


Number of apps: 
1
Total entries:
5
Total entries:
5
Total entries:
15
Total entries:
15


In [2]:
test_graph_df


,source,target,action,widget_id,widget,strategy,app
0,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.BuyProActivity,back,NaN,NaN,systematic,com.antonnikitin.solunarforecast
1,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.BuyProActivity,clickMenuItem,2.131624e+09,com.android.internal.view.menu.ActionMenuItemView,systematic,com.antonnikitin.solunarforecast
2,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.BuyProActivity,click,1.690831e+07,android.widget.Button,systematic,com.antonnikitin.solunarforecast
3,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.BuyProActivity,click,1.690831e+07,android.widget.Button,random,com.antonnikitin.solunarforecast
4,com.antonnikitin.solunarforecast.BuyProActivity,com.antonnikitin.solunarforecast.MainActivity,back,NaN,NaN,random,com.antonnikitin.solunarforecast


In [17]:
stat_graph_df

,package,source,target,method,widget,app
0,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.BuyProActivity,com.antonnikitin.solunarforecast.MainActivity,onClick,android.widget.Button,com.antonnikitin.solunarforecast
1,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.WeatherFragment,com.antonnikitin.solunarforecast.SelectLocatio...,onOptionsItemSelected,android.view.MenuItem,com.antonnikitin.solunarforecast
2,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.WeatherFragment,onClick,NaN,com.antonnikitin.solunarforecast
3,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.WeatherFragment,onCreate,NaN,com.antonnikitin.solunarforecast
4,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.ForecastFragment,onActivityResult,NaN,com.antonnikitin.solunarforecast
5,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.SelectLocatio...,onOptionsItemSelected,android.view.MenuItem,com.antonnikitin.solunarforecast
6,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.CalendarFragment,onClick,android.widget.LinearLayout,com.antonnikitin.solunarforecast
7,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.BuyProActivity,com.antonnikitin.solunarforecast.ForecastFragment,onClick,NaN,com.antonnikitin.solunarforecast
8,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.ForecastFragment,onCreate,NaN,com.antonnikitin.solunarforecast
9,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.CalendarFragment,onCreate,NaN,com.antonnikitin.solunarforecast


In [16]:
stat_trans_graph_df

,package,source,target,method,widget,app
0,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.BuyProActivity,com.antonnikitin.solunarforecast.MainActivity,onClick,android.widget.Button,com.antonnikitin.solunarforecast
1,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.WeatherFragment,com.antonnikitin.solunarforecast.SelectLocatio...,onOptionsItemSelected,android.view.MenuItem,com.antonnikitin.solunarforecast
2,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.WeatherFragment,onClick,NaN,com.antonnikitin.solunarforecast
3,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.WeatherFragment,onCreate,NaN,com.antonnikitin.solunarforecast
4,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.ForecastFragment,onActivityResult,NaN,com.antonnikitin.solunarforecast
5,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.SelectLocatio...,onOptionsItemSelected,android.view.MenuItem,com.antonnikitin.solunarforecast
6,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.CalendarFragment,onClick,android.widget.LinearLayout,com.antonnikitin.solunarforecast
7,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.BuyProActivity,com.antonnikitin.solunarforecast.ForecastFragment,onClick,NaN,com.antonnikitin.solunarforecast
8,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.ForecastFragment,onCreate,NaN,com.antonnikitin.solunarforecast
9,com.antonnikitin.solunarforecast,com.antonnikitin.solunarforecast.MainActivity,com.antonnikitin.solunarforecast.CalendarFragment,onCreate,NaN,com.antonnikitin.solunarforecast


In [8]:
unique_widgets_per_app = test_graph_df.groupby(["strategy", "app"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count")
unique_widgets_per_app['percentage'] = unique_widgets_per_app.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
unique_widgets_per_app


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
      <th>percentage</th>
    </tr>
    <tr>
      <th>strategy</th>
      <th>app</th>
      <th>widget</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="2" valign="top">random</th>
      <th rowspan="2" valign="top">com.antonnikitin.solunarforecast</th>
      <th>NaN</th>
      <td>1</td>
      <td>50.000000</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>1</td>
      <td>50.000000</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">systematic</th>
      <th rowspan="3" valign="top">com.antonnikitin.solunarforecast</th>
      <th>NaN</th>
      <td>1</td>
      <td>33.333333</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>1</td>
      <td>33.333333</td>
    </tr>
    <tr>
      <th>com.android.internal.view.menu.ActionMenuItemView</th>
      <td>1</td>
      <td>33.333333</td>
    </tr>
  </tbody>
</table>
</div>

In [15]:
number_of_unique_screens_per_app = test_graph_df[['strategy', 'app', 'source', 'target', 'widget']].melt(
    id_vars=['strategy', 'app', 'widget'])
number_of_unique_screens_per_app

,strategy,app,widget,variable,value
0,systematic,com.antonnikitin.solunarforecast,NaN,source,com.antonnikitin.solunarforecast.MainActivity
1,systematic,com.antonnikitin.solunarforecast,com.android.internal.view.menu.ActionMenuItemView,source,com.antonnikitin.solunarforecast.MainActivity
2,systematic,com.antonnikitin.solunarforecast,android.widget.Button,source,com.antonnikitin.solunarforecast.MainActivity
3,random,com.antonnikitin.solunarforecast,android.widget.Button,source,com.antonnikitin.solunarforecast.MainActivity
4,random,com.antonnikitin.solunarforecast,NaN,source,com.antonnikitin.solunarforecast.BuyProActivity
5,systematic,com.antonnikitin.solunarforecast,NaN,target,com.antonnikitin.solunarforecast.BuyProActivity
6,systematic,com.antonnikitin.solunarforecast,com.android.internal.view.menu.ActionMenuItemView,target,com.antonnikitin.solunarforecast.BuyProActivity
7,systematic,com.antonnikitin.solunarforecast,android.widget.Button,target,com.antonnikitin.solunarforecast.BuyProActivity
8,random,com.antonnikitin.solunarforecast,android.widget.Button,target,com.antonnikitin.solunarforecast.BuyProActivity
9,random,com.antonnikitin.solunarforecast,NaN,target,com.antonnikitin.solunarforecast.MainActivity
